In [1]:
import pandas as pd

In [2]:
stocks = pd.read_csv("stocks-df.csv")
industries = pd.read_csv("industry-data.csv")

In [7]:
print(f"Containing Stock Data from {stocks['Date'].min()} to {stocks['Date'].max()}")

Containing Stock Data from 1990-01-02 to 2022-01-11


In [11]:
stocks = stocks.merge(industries, how="left", left_on="Stock", right_on="symbol")
stocks

,Date,Stock,Open,High,Low,Close,Volume,Change Pct,Unnamed: 0,symbol,industry
0,2013-01-02,20MICRONS,80.35,80.55,77.50,77.80,1088880,-3.23,0.0,20MICRONS,Mining/Minerals
1,2013-01-03,20MICRONS,78.58,79.78,76.00,76.40,590180,-1.80,0.0,20MICRONS,Mining/Minerals
2,2013-01-04,20MICRONS,76.50,77.25,75.50,75.83,380850,-0.75,0.0,20MICRONS,Mining/Minerals
3,2013-01-07,20MICRONS,76.88,78.00,74.00,74.90,1144204,-1.23,0.0,20MICRONS,Mining/Minerals
4,2013-01-08,20MICRONS,74.97,75.97,74.25,75.28,605020,0.51,0.0,20MICRONS,Mining/Minerals
...,...,...,...,...,...,...,...,...,...,...,...
5440147,2022-01-07,ZYDUSWELL,1866.00,1879.00,1840.00,1844.65,27888,-1.35,2022.0,ZYDUSWELL,Pharmaceuticals
5440148,2022-01-10,ZYDUSWELL,1850.25,1865.00,1840.10,1859.15,11310,0.79,2009.0,ZYDUSWELL,Pharmaceuticals
5440149,2022-01-10,ZYDUSWELL,1850.25,1865.00,1840.10,1859.15,11310,0.79,2022.0,ZYDUSWELL,Pharmaceuticals
5440150,2022-01-11,ZYDUSWELL,1865.00,1868.60,1848.00,1853.15,6106,-0.32,2009.0,ZYDUSWELL,Pharmaceuticals


In [12]:
stocks.to_csv("stocks-with-industry.csv")

In [2]:
import pandas as pd
stocks = pd.read_csv("stocks-with-industry.csv")

In [6]:
stocks = stocks.set_index("Date").drop(columns="symbol")

In [12]:
from tqdm.auto import tqdm
company_columns_dict = {}

for company_symbol in tqdm(stocks.Stock.unique()):
    company_columns_dict[company_symbol] = stocks[stocks.Stock == "20MICRONS"]["Open"]

  0%|          | 0/1055 [00:00<?, ?it/s]

In [16]:
stocks = pd.DataFrame(company_columns_dict)

In [17]:
stocks.to_csv("company-datewise.csv")

In [1]:
import pandas as pd
pd.read_csv("/Users/irsh/Documents/workdir/nse2vec/data/embedding_data_files/Data/historical_stocks.csv").sector.unique()

array(['FINANCE', 'CONSUMER SERVICES', 'TECHNOLOGY', 'PUBLIC UTILITIES',
       'CAPITAL GOODS', 'BASIC INDUSTRIES', 'HEALTH CARE',
       'CONSUMER DURABLES', nan, 'ENERGY', 'MISCELLANEOUS', 'SECTOR',
       'TRANSPORTATION', 'CONSUMER NON-DURABLES'], dtype=object)

In [2]:
industry_df = pd.read_csv("/Users/irsh/Documents/workdir/nse2vec/data/industry-data.csv")
len(set([z[0] for z in industry_df.industry.str.split(" ")]))

112

In [3]:
industry_df

,Unnamed: 0,symbol,industry
0,0,20MICRONS,Mining/Minerals
1,1,360ONE,Finance & Investments
2,2,3MINDIA,Diversified
3,3,3PLAND,Finance & Investments
4,4,3IINFOLTD,IT Consulting & Software
...,...,...,...
2021,2021,ZYDUSLIFE,Pharmaceuticals
2022,2022,ZYDUSWELL,Pharmaceuticals
2023,2023,ECLERX,IT Enabled Services
2024,2024,EMUDHRA,IT Enabled Services


In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
chat = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
    groq_api_key="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"
)

In [11]:
from langchain_core import pydantic_v1
class Sector(pydantic_v1.BaseModel):
    sector: str = pydantic_v1.Field(description="One category amongst ['FINANCE', 'CONSUMER SERVICES', 'TECHNOLOGY', 'PUBLIC UTILITIES', 'CAPITAL GOODS', 'BASIC INDUSTRIES', 'HEALTH CARE','CONSUMER DURABLES', 'ENERGY','TRANSPORTATION', 'CONSUMER NON-DURABLES']")

system = """\
You are a bot only allowed to output one of ['FINANCE', 'CONSUMER SERVICES', 'TECHNOLOGY', 'PUBLIC UTILITIES',
       'CAPITAL GOODS', 'BASIC INDUSTRIES', 'HEALTH CARE',
       'CONSUMER DURABLES', 'ENERGY',
       'TRANSPORTATION', 'CONSUMER NON-DURABLES'] whichever seems the closest to the category in the input\
"""

prompt_template = ChatPromptTemplate.from_messages([("system", system), ("human", "Tell me which is the most similar category amongst ['FINANCE', 'CONSUMER SERVICES', 'TECHNOLOGY', 'PUBLIC UTILITIES', 'CAPITAL GOODS', 'BASIC INDUSTRIES', 'HEALTH CARE','CONSUMER DURABLES', 'ENERGY','TRANSPORTATION', 'CONSUMER NON-DURABLES'] to this one: {text}")])

chain = prompt_template | chat.with_structured_output(Sector)


In [27]:
from tqdm.auto import tqdm
import time

industry_sector_map = {'Mining/Minerals': 'BASIC INDUSTRIES',
 'Finance & Investments': 'FINANCE',
 'Diversified': 'CONSUMER DURABLES',
 'IT Consulting & Software': 'TECHNOLOGY',
 'Engineering - General': 'CAPITAL GOODS',
 'Electric Equipment - General': 'CAPITAL GOODS',
 'Trading & Distribution': 'CONSUMER DURABLES',
 'Cement': 'BASIC INDUSTRIES',
 'Food Processing - Bakery/Dairy/Fruits/Others': 'CONSUMER NON-DURABLES',
 'Packaging & Containers': 'CONSUMER DURABLES',
 'Financial Technologies (Fintech)': 'FINANCE',
 'Castings/Foundry': 'BASIC INDUSTRIES',
 'Realty': 'FINANCE',
 'Steel - Tubes/Pipes': 'BASIC INDUSTRIES',
 'Construction & Engineering': 'CAPITAL GOODS',
 'Auto Ancl - Others': 'CONSUMER DURABLES',
 'Logistics - Warehousing/Supply Chain/Others': 'TRANSPORTATION',
 'Agricultural Products': 'CONSUMER NON-DURABLES',
 'IT Enabled Services': 'TECHNOLOGY',
 'Textiles - Processing/Texturising': 'CONSUMER DURABLES',
 'Finance - Housing': 'FINANCE',
 'Oil Drilling And Exploration': 'ENERGY'}

for ind in tqdm(industry_df.industry.unique()[22:]):
    print(f"Processing {ind}", end="\t\t\t\t\r")
    industry_sector_map[ind] = chain.invoke(ind).sector
    # time.sleep(1)

  0%|          | 0/162 [00:00<?, ?it/s]

In [28]:
len(industry_sector_map)

184

In [31]:
def industry_to_sector(x):
    return industry_sector_map[x]


In [ ]:
industry_sector_map

In [34]:
industry_df['sector'] = industry_df['industry'].apply(industry_to_sector)

In [42]:
pd.DataFrame({"industry": industry_sector_map.keys(), "sector": industry_sector_map.values()}).to_csv("industry-sector-map.csv")

In [44]:
industry_df.drop(columns="Unnamed: 0").to_csv("industry-data.csv")

In [67]:
import pandas as pd
pd.read_csv("embedding_data_files/Data/historical_stocks.csv")

,ticker,exchange,name,sector,industry
0,PIH,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
1,PIHPP,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
2,TURN,NASDAQ,180 DEGREE CAPITAL CORP.,FINANCE,FINANCE/INVESTORS SERVICES
3,FLWS,NASDAQ,"1-800 FLOWERS.COM, INC.",CONSUMER SERVICES,OTHER SPECIALTY STORES
4,FCCY,NASDAQ,1ST CONSTITUTION BANCORP (NJ),FINANCE,SAVINGS INSTITUTIONS
...,...,...,...,...,...
6455,ZOES,NYSE,"ZOE&#39;S KITCHEN, INC.",CONSUMER SERVICES,RESTAURANTS
6456,ZTS,NYSE,ZOETIS INC.,HEALTH CARE,MAJOR PHARMACEUTICALS
6457,ZTO,NYSE,ZTO EXPRESS (CAYMAN) INC.,TRANSPORTATION,TRUCKING FREIGHT/COURIER SERVICES
6458,ZUO,NYSE,"ZUORA, INC.",TECHNOLOGY,COMPUTER SOFTWARE: PREPACKAGED SOFTWARE


In [68]:
pd.read_csv("embedding_data_files/Data/returns_df_611.csv")

,Unnamed: 0,AAPL,AB,ABM,ABT,ABX,ACXM,ADBE,ADI,ADM,...,WTS,WWW,WY,XCRA,XLNX,XOM,XRAY,XRX,ZION,ZIXI
0,2000-01-03,0.088754,-0.006263,-0.003067,-0.036145,0.000000,0.041667,-0.025093,-0.030242,-0.010309,...,-0.029661,-0.057143,-0.026980,-0.050279,0.028180,-0.027929,-0.005291,0.066116,-0.062302,-0.061514
1,2000-01-04,-0.084310,-0.004202,0.003077,-0.028571,-0.014134,-0.005000,-0.083889,-0.050589,-0.010417,...,0.000000,0.000000,-0.037567,-0.052941,-0.022727,-0.019154,0.000000,-0.046512,-0.048423,-0.018487
2,2000-01-05,0.014634,0.004219,-0.012270,-0.001838,0.017921,0.010050,0.019771,0.014599,-0.015789,...,-0.008734,-0.024242,0.052974,0.043478,-0.021888,0.054516,0.017285,0.051490,-0.001183,0.021403
3,2000-01-06,-0.086538,0.016807,0.003106,0.034991,0.000000,0.009950,0.008163,-0.027338,0.005348,...,0.008811,0.000000,0.046778,-0.062500,-0.086713,0.051698,0.001309,-0.020619,0.014218,-0.054483
4,2000-01-07,0.047368,0.033058,0.003096,0.010676,-0.024648,0.004926,0.048583,0.028107,0.015957,...,-0.021834,0.024845,-0.032884,0.047619,0.113323,-0.002935,-0.002611,0.023684,0.002336,0.078014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4687,2018-08-20,-0.009744,0.006723,0.013815,-0.001236,0.011976,0.005188,0.016524,-0.008881,0.004352,...,0.001804,0.001046,-0.004488,0.000700,0.001408,0.007028,0.004913,0.011507,0.004653,-0.012590
4688,2018-08-21,-0.001949,0.001669,0.020440,0.001702,0.003945,0.012118,0.006967,0.023256,-0.000591,...,0.014406,0.005486,-0.006763,0.004199,0.018279,0.000254,0.002831,0.020550,0.004446,0.009108
4689,2018-08-22,0.000047,0.001667,0.000910,0.014208,0.019646,0.008426,0.016064,0.020851,-0.004730,...,-0.011834,0.001299,0.001418,0.003484,0.007595,0.014335,-0.001796,-0.005753,-0.002767,-0.007220
4690,2018-08-23,0.002046,-0.003328,-0.079745,0.002436,-0.014451,-0.004837,0.005713,-0.005413,0.000198,...,-0.004192,0.000000,-0.020397,-0.004167,0.002604,-0.011005,-0.020051,-0.007233,-0.009062,-0.007273


In [69]:
historical_stocks=pd.read_csv("industry-data.csv")
historical_stocks["ticker"] = historical_stocks["symbol"]
historical_stocks = historical_stocks.drop(columns=["Unnamed: 0", "symbol"])
historical_stocks

,industry,sector,ticker
0,Mining/Minerals,BASIC INDUSTRIES,20MICRONS
1,Finance & Investments,FINANCE,360ONE
2,Diversified,CONSUMER DURABLES,3MINDIA
3,Finance & Investments,FINANCE,3PLAND
4,IT Consulting & Software,TECHNOLOGY,3IINFOLTD
...,...,...,...
2021,Pharmaceuticals,HEALTH CARE,ZYDUSLIFE
2022,Pharmaceuticals,HEALTH CARE,ZYDUSWELL
2023,IT Enabled Services,TECHNOLOGY,ECLERX
2024,IT Enabled Services,TECHNOLOGY,EMUDHRA


In [70]:
historical_stocks = historical_stocks.loc[historical_stocks.index.repeat(2)].reset_index(drop=True)[["ticker", "sector", "industry"]]

In [34]:
for i in range(len(historical_stocks)):
    if(i%2==0):
        historical_stocks.loc[i, "ticker"] += ".NS"
    else:
        historical_stocks.loc[i, "ticker"] += ".BO"

In [35]:
historical_stocks.to_csv("nse_historical_stocks.csv")

In [81]:
historical_stocks = pd.read_csv("historical-stocks.csv", index_col=None)

In [82]:
list(historical_stocks.ticker)

['20MICRONS.NS',
 '20MICRONS.BO',
 '360ONE.NS',
 '360ONE.BO',
 '3MINDIA.NS',
 '3MINDIA.BO',
 '3PLAND.NS',
 '3PLAND.BO',
 '3IINFOLTD.NS',
 '3IINFOLTD.BO',
 '5PAISA.NS',
 '5PAISA.BO',
 '63MOONS.NS',
 '63MOONS.BO',
 'A2ZINFRA.NS',
 'A2ZINFRA.BO',
 'AAATECH.NS',
 'AAATECH.BO',
 'ABB.NS',
 'ABB.BO',
 'ABMINTLLTD.NS',
 'ABMINTLLTD.BO',
 'ACC.NS',
 'ACC.BO',
 'ADFFOODS.NS',
 'ADFFOODS.BO',
 'AGI.NS',
 'AGI.BO',
 'AGSTRA.NS',
 'AGSTRA.BO',
 'AIAENG.NS',
 'AIAENG.BO',
 'GOLDTECH.NS',
 'GOLDTECH.BO',
 'AKG.NS',
 'AKG.BO',
 'AMDIND.NS',
 'AMDIND.BO',
 'AMJLAND.NS',
 'AMJLAND.BO',
 'APLAPOLLO.NS',
 'APLAPOLLO.BO',
 'ARSSINFRA.NS',
 'ARSSINFRA.BO',
 'ARTNIRMAN.NS',
 'ARTNIRMAN.BO',
 'ASKAUTOLTD.NS',
 'ASKAUTOLTD.BO',
 'AVG.NS',
 'AVG.BO',
 'AVTNPL.NS',
 'AVTNPL.BO',
 'AXISCADES.NS',
 'AXISCADES.BO',
 'AYMSYNTEX.NS',
 'AYMSYNTEX.BO',
 'AADHARHFC.NS',
 'AADHARHFC.BO',
 'AAKASH.NS',
 'AAKASH.BO',
 'AAREYDRUGS.NS',
 'AAREYDRUGS.BO',
 'AARON.NS',
 'AARON.BO',
 'AARTECH.NS',
 'AARTECH.BO',
 'AARTIDRUGS.N

In [83]:
import os
from tqdm import tqdm

stocks_to_use = set(pd.read_csv("company-datewise.csv").columns)

file_names = sorted([csv_file for csv_file in os.listdir("kaggle-data") if csv_file[:-4] in list(historical_stocks.ticker)]) # and csv_file[-6:-4]=='NS'])

In [84]:
len(file_names)

1587

In [85]:

list_dates = []
chosen_file_names = []
for file_name in tqdm(file_names):
    s = pd.read_csv("kaggle-data/"+file_name)
    s['Date'] = pd.to_datetime(s['Date'])

    s = s.sort_values(by="Date")
    if(s.loc[0, 'Date'] < pd.Timestamp(year=2010, month=1, day=1)):
        list_dates.append(set(s['Date']))
        chosen_file_names.append(file_name)

100%|██████████| 1587/1587 [00:15<00:00, 104.40it/s]


In [86]:
len(list_dates)

1164

In [87]:
min(sorted(set.intersection(*list_dates)))

Timestamp('2012-06-18 00:00:00')

In [88]:
df_returns = pd.DataFrame({"Date": sorted(set.intersection(*list_dates))})

In [89]:
df_returns

,Date
0,2012-06-18
1,2012-06-19
2,2012-06-20
3,2012-06-21
4,2012-06-22
...,...
1946,2020-06-01
1947,2020-06-02
1948,2020-06-03
1949,2020-06-04


In [90]:
for file_name in tqdm(chosen_file_names):
    s = pd.read_csv("kaggle-data/"+file_name)
    s['Date'] = pd.to_datetime(s['Date'])
    shifted_s = s['Close'].shift(1).iloc[1:]
    s[file_name[:-4]] = (s['Close'].iloc[1:].subtract(shifted_s)).divide(shifted_s)

    df_returns = df_returns.merge(s[[file_name[:-4], 'Date']], left_on="Date", right_on="Date")
    
df_returns

100%|██████████| 1164/1164 [00:18<00:00, 64.36it/s]


,Date,20MICRONS.BO,20MICRONS.NS,3MINDIA.BO,3MINDIA.NS,63MOONS.BO,63MOONS.NS,AARTIDRUGS.BO,AARTIDRUGS.NS,AARTIIND.BO,...,VSTIND.NS,VSTTILLERS.BO,VTL.BO,VTL.NS,ZENSARTECH.BO,ZENSARTECH.NS,ZODIACLOTH.BO,ZODIACLOTH.NS,ZYDUSWELL.BO,ZYDUSWELL.NS
0,2012-06-18,-0.004002,0.003452,-0.001315,0.016390,-0.011660,-0.010499,0.038618,0.033535,-0.016540,...,-0.010658,-0.000967,-0.019067,-0.016518,-0.005152,-0.004297,0.024868,0.028716,-0.005195,-0.006755
1,2012-06-19,-0.023536,-0.024656,0.001914,0.004741,0.044613,0.042128,-0.026223,-0.032447,0.003504,...,0.005943,-0.011720,-0.017151,-0.023604,0.001726,0.000432,-0.020650,-0.023520,0.007180,0.004839
2,2012-06-20,0.000588,-0.004115,-0.005730,-0.005192,-0.008302,-0.006663,-0.015273,-0.008889,0.020251,...,0.000625,0.003047,-0.003490,-0.003022,-0.010987,-0.006040,-0.012915,-0.002382,0.006999,0.010543
3,2012-06-21,0.008226,0.019481,0.002711,0.005749,0.010031,0.009270,0.000816,0.006523,0.004107,...,0.003265,-0.005424,0.020546,0.010026,0.010237,0.010200,-0.019493,-0.022818,0.005149,0.005410
4,2012-06-22,-0.005245,-0.013318,-0.005258,-0.011838,0.044056,0.044952,-0.021207,-0.021466,-0.008862,...,0.009451,0.012542,-0.011668,-0.003232,0.012505,0.010097,-0.008442,0.001358,-0.002049,-0.000769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4000,2020-06-01,0.042308,0.028681,-0.012695,-0.013236,0.046618,0.048358,-0.018484,-0.017125,0.005893,...,0.076756,-0.002983,0.017436,0.021065,0.031411,0.035118,0.057702,0.066794,0.008067,0.009298
4001,2020-06-02,0.014760,0.022305,-0.004367,-0.005001,0.029694,0.013925,-0.006452,-0.006697,-0.007998,...,-0.010854,0.200000,0.065928,0.057794,0.044297,0.049499,0.045987,0.031306,0.028128,0.025581
4002,2020-06-03,0.058182,0.049091,-0.004876,-0.003089,-0.007634,0.003433,0.044139,0.040508,-0.017870,...,0.017109,0.000317,0.028012,0.031402,0.017497,0.018548,0.032759,0.035126,0.014488,0.014997
4003,2020-06-04,-0.018900,-0.019064,0.009872,0.009039,-0.009402,-0.005988,0.015524,0.017820,-0.059709,...,-0.021341,0.124700,0.011336,0.010581,0.049505,0.049948,-0.041736,-0.052786,-0.012268,-0.013104


In [91]:
df_returns = df_returns.drop(columns=(df_returns.isna().sum()[df_returns.isna().sum() != 0].index))

In [92]:
df_returns = df_returns.ffill()

In [93]:
df_returns.to_csv("nsenbse-data.csv")

In [66]:
df_returns

,Date,20MICRONS.NS,3MINDIA.NS,63MOONS.NS,AARTIDRUGS.NS,AARTIIND.NS,AARVEEDEN.NS,ABAN.NS,ABBOTINDIA.NS,ACC.NS,...,VINATIORGA.NS,VINDHYATEL.NS,VINYLINDIA.NS,VISAKAIND.NS,VISASTEEL.NS,VSTIND.NS,VTL.NS,ZENSARTECH.NS,ZODIACLOTH.NS,ZYDUSWELL.NS
0,2012-06-18,0.003452,0.016390,-0.010499,0.033535,-0.015204,-0.024674,0.021948,-0.016367,0.001880,...,-0.024413,-0.019538,0.047847,0.000000,-0.004902,-0.010658,-0.016518,-0.004297,0.028716,-0.006755
1,2012-06-19,-0.024656,0.004741,0.042128,-0.032447,0.009123,0.049107,-0.009966,0.002019,0.001958,...,-0.016525,0.027536,-0.018265,-0.037804,0.000000,0.005943,-0.023604,0.000432,-0.023520,0.004839
2,2012-06-20,-0.004115,-0.005192,-0.006663,-0.008889,0.013908,-0.015603,0.006097,-0.008129,0.035221,...,-0.024004,-0.035614,0.004651,0.042519,-0.000985,0.000625,-0.003022,-0.006040,-0.002382,0.010543
3,2012-06-21,0.019481,0.005749,0.009270,0.006523,0.005487,0.020173,-0.005919,0.007425,-0.013767,...,-0.002459,0.037660,-0.018519,0.023232,0.000986,0.003265,0.010026,0.010200,-0.022818,0.005410
4,2012-06-22,-0.013318,-0.011838,0.044952,-0.021466,-0.006139,0.011299,-0.001701,0.000800,-0.030829,...,0.029093,-0.028894,-0.004717,0.039354,0.000000,0.009451,-0.003232,0.010097,0.001358,-0.000769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,2020-06-01,0.028681,-0.013236,0.048358,-0.017125,0.006761,0.195266,0.038568,0.006111,0.010668,...,0.025477,0.049970,0.000928,0.049796,0.000000,0.076756,0.021065,0.035118,0.066794,0.009298
2012,2020-06-02,0.022305,-0.005001,0.013925,-0.006697,-0.009311,0.198020,0.013263,0.003453,0.002610,...,-0.005851,-0.039918,-0.063948,0.047989,0.047059,-0.010854,0.057794,0.049499,0.031306,0.025581
2013,2020-06-03,0.049091,-0.003089,0.003433,0.040508,-0.018642,0.099174,0.049738,-0.009856,-0.020045,...,-0.002460,0.016920,0.002970,0.048438,0.044944,0.017109,0.031402,0.018548,0.035126,0.014997
2014,2020-06-04,-0.019064,0.009039,-0.005988,0.017820,-0.058663,-0.026316,0.044888,-0.002040,0.006184,...,-0.011269,0.000000,0.009872,0.046453,0.032258,-0.021341,0.010581,0.049948,-0.052786,-0.013104


In [74]:
import pandas as pd
import sklearn.preprocessing

returns = pd.read_csv("returns_df_528_nse.csv").drop(columns="Unnamed: 0")
returns[returns.columns[1:]] = sklearn.preprocessing.normalize(returns.drop(columns="Date"))

In [75]:
returns.to_csv("returns_df_528_nse_norm.csv")